In [1]:
# VGG 16
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model

In [2]:
# VGG is made up of conv blocks which are of variable filter sizes
def conv_block(input, number_filters):
    x = Conv2D (filters =number_filters, kernel_size =3, padding ='same', activation='relu')(input)
    x = Conv2D (filters =number_filters, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =number_filters, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    return x

In [3]:
# creating full model
input = Input(shape =(224,224,3)) # need to change to actual input
x = conv_block(input, 64)
x = conv_block(x, 128)
x = conv_block(x, 256)
x = conv_block(x, 512)
x = conv_block(x, 512)
x = conv_block(x, 512)

# fully connected layers at the end
x = Flatten()(x) 
x = Dense(units = 4096, activation ='relu')(x) 
x = Dense(units = 4096, activation ='relu')(x) 
output = Dense(units = 4, activation ='softmax')(x)

In [4]:
model = Model (inputs=input, outputs =output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     73856     
                                                             

In [5]:
import os
import pandas as pd
from tqdm.auto import tqdm
import cv2
# importing the training data

# loading in images and turing it into an array
path = os.getcwd()

path = os.path.join(path, 'dataset')
df_labels = pd.read_csv(os.path.join(path, 'label.csv'))

# getting train and test dataframes
train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

classes = {'meningioma_tumor': 0, 'no_tumor': 1, 'glioma_tumor': 2, 'pituitary_tumor': 3}

def feature_extraction(dataset, img_path, name):
    images = []
    labels = []
    for row in tqdm(dataset.iterrows(), total=dataset.shape[0], desc="Progress"):
        image_name = row[1][0]
        label = row[1][1]
        # opening and flattening image
        img = cv2.imread(os.path.join(img_path, image_name))
        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
        labels.append(classes[label])
        images.append(img)

    return images, labels

train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')

x_train, y_train = feature_extraction(train, train_path, 'train')
x_test, y_test = feature_extraction(test, test_path, 'test')

print('Train size: ', len(x_train))
print('Test size: ', len(x_test))

Progress: 100%|██████████| 900/900 [00:05<00:00, 172.54it/s]

Train size:  2100
Test size:  900


In [6]:
# compiling the model and training

import tensorflow as tf
import numpy as np

y_train = tf.one_hot(y_train, depth=len(classes))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

history = model.fit(
    x=np.array(x_train), y=np.array(y_train), batch_size=64, epochs=10
)


Epoch 1/10
33/33 [==============================] - 2398s 72s/step - loss: 461.6220 - binary_accuracy: 0.7408 - false_negatives: 2039.0000
Epoch 2/10


In [1]:
x_test = np.array(x_test)
y_test = tf.one_hot(y_train, depth=len(classes))

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


NameError: name 'np' is not defined